In [1]:
from sub_func import *

/opt/anaconda3/envs/finTF/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
### 메타 정보 생성

# 투자가 진행될 시점
curr_year = '2022'
curr_quarter = 'Q1'

# 종가 추출(start_date, end_date) 및 기타 재무 정보 확보(target_year, target_quarter)를 위한 날짜
if curr_quarter == 'Q1':
    start_date = f"{int(curr_year)-1}1001"
    end_date = f"{int(curr_year)-1}1231"
    target_year = f"{int(curr_year)-1}"
    target_quarter = "Q4"
elif curr_quarter == 'Q2':
    start_date = f"{curr_year}0101"
    end_date = f"{curr_year}0331"
    target_year = curr_year
    quarter_num = curr_quarter[-1]
    target_quarter = f"Q{int(quarter_num)-1}"
elif curr_quarter == 'Q3':
    start_date = f"{curr_year}0401"
    end_date = f"{curr_year}0630"
    target_year = curr_year
    quarter_num = curr_quarter[-1]
    target_quarter = f"Q{int(quarter_num)-1}"
elif curr_quarter == 'Q4':
    start_date = f"{curr_year}0701"
    end_date = f"{curr_year}0930"
    target_year = curr_year
    quarter_num = curr_quarter[-1]
    target_quarter = f"Q{int(quarter_num)-1}"

# 포트폴리오 내 종목 개수
n = 10

# 초기 투자 자본 (₩1,000,000)
init_balance = 1000000

# 포트폴리오 종목 불러오기
portfolio = get_current_quarter_pf(curr_year, curr_quarter, n)

# 각 포트폴리오의 가격 비중 산정
portfolio['max_price_allowed'] = round(portfolio['Weight']*init_balance)

# 각 포트폴리오 종목에 대해 최대 보유 개수 구하여 portfolio df에 추가
n_stock_list = []
for i in range(len(portfolio)):
    ticker = portfolio['Ticker'].iloc[i]

    # 가격 데이터 불러와서 종가만 남기고
    price_df = stock_price_info(ticker, start_date, end_date)
    close_price = price_df.iloc[-1]['Close']

    max_price_allowed = portfolio['max_price_allowed'].iloc[i]
    n_stock_list.append(max_price_allowed // close_price)

# 개수 구하기
portfolio['n_stock'] = n_stock_list

/opt/anaconda3/envs/finTF/lib/python3.10/site-packages/scipy/optimize/_slsqp_py.py:434: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)


In [3]:
def t1_analyst_common(target_year, start_date, end_date):

    # === 국제 뉴스 관련 정보 수집 === #
    intl_news_df = intl_news_info(target_year, start_date, end_date)

    SA_result_intl_news_dict = {
        'positive': [],
        'negative': []
    }

    for i in range(len(intl_news_df)):
        news_title = intl_news_df.iloc[i]['news_title']
        SA_result = get_SA_result(news_title)

        if SA_result['label'] == 'positive' and SA_result['prob'] > 0.9:
            SA_result_intl_news_dict['positive'].append(news_title)
        elif SA_result['label'] == 'negative' and SA_result['prob'] > 0.9:
            SA_result_intl_news_dict['negative'].append(news_title)

    print('='*50)
    print('intl_news 정보 취합 완료!')

    # === 거시 경제 관련 정보 수집 === #
    def create_macro_econ_dict(country, econ_item):
        result_dict = macro_econ_dict[country][econ_item].set_index('Date').to_dict('index')
        final_dict = {k: v[list(v.keys())[0]] for k, v in result_dict.items()}

        return final_dict
    
    macro_econ_dict = macro_econ_info(target_year, start_date, end_date)
    country_list = list(macro_econ_dict.keys())

    reports_dict = {}

    for country in country_list:

        # ======================================================================================== #
        system_macro_econ = f"""당신은 증권회사에 고용된 국제관계 전문가입니다.
        주식투자의 관점에서 주어진 {country} 국가에 대한 경제지표를 분석하고, 이에 대한 의견을 알려주세요.
        의견을 개진할 때에는 반드시에 경제지표에 대한 분석에 기반한 근거를 제시해야 합니다.
        ** {country}에 투자하는 것이 아닌, 한국 주식 투자에 대한 분석을 진행해야 합니다. **
        ** 해당 국가의 경제 동향이 어떠한 영향을 줄지 분석하세요. **

        응답은 반드시 markdown 문법에 따라 다음 구조로 작성되어야 합니다.

        1. {country} 경제에 대한 분석
        2. 투자의견
        """

        prompt_macro_econ = f""""""
        # ======================================================================================== #

        target_country_dict = macro_econ_dict[country]
        econ_items = list(target_country_dict.keys())

        for econ_item in econ_items:
            final_dict = create_macro_econ_dict(country, econ_item)
            prompt_macro_econ += f"{econ_item}: {final_dict}\n"

        macro_econ_response = to_GPT(system_macro_econ, prompt_macro_econ)['choices'][0]['message']['content']
        reports_dict[country] = macro_econ_response
        print('-'*50)
        print(f'{country}에 대한 보고서 생성 완료!')

    return {
        'SA_result_intl_news_dict': SA_result_intl_news_dict,
        'reports_dict': reports_dict
    }

In [4]:
def t1_analyst_corp(ticker, target_year, start_date, end_date):

    fin_statement = None
    fin_report = None
    SA_result_corp_rel_news_dict = None

    failed_item = []

    # === 재무제표 관련 정보 취합 === #
    try:
        fin_statement = fin_statement_info(ticker, target_year, target_quarter)
    except Exception as e:
        failed_item.append('재무제표')

    # === 재무보고서 분석 취합 === #
    system_fin_report = """당신은 증권회사에 고용된 재무전문가입니다. 
    주식투자의 관점에서 본 재무보고서를 요약하고, 이에 대한 의견을 알려주세요. 
    의견을 개진할 때에는 반드시 보고서에서 근거를 제시해야 합니다.

    응답은 반드시 markdown 문법에 따라 다음 구조로 작성되어야 합니다.

    1. 재무보고서 요약
    2. 투자의견
    """

    try:
        fin_report = reports_info(ticker, target_year, target_quarter)
        fin_report_text = fin_report['1. 요약재무정보.csv'][0][4:-4]

        prompt_fin_report = f"{fin_report_text}"
        result = to_GPT(system_fin_report, prompt_fin_report)
        fin_report = result['choices'][0]['message']['content']
    except Exception as e:
        failed_item.append('재무보고서')

    # === 기업 관련 뉴스 취합 === #
    try: 
        corp_rel_news_df = corp_rel_news_info(ticker, target_year, start_date, end_date)
        corp_rel_news_df = corp_rel_news_df[corp_rel_news_df['news_category'].str.contains('증권')]

        # 활용할 최종 정보!
        SA_result_corp_rel_news_dict = {
            'positive': [],
            'negative': []
        }

        for i in range(len(corp_rel_news_df)):
            news_title = corp_rel_news_df.iloc[i]['news_title']
            SA_result = get_SA_result(news_title)

            if SA_result['label'] == 'positive' and SA_result['prob'] > 0.9:
                SA_result_corp_rel_news_dict['positive'].append(news_title)
            elif SA_result['label'] == 'negative' and SA_result['prob'] > 0.9:
                SA_result_corp_rel_news_dict['negative'].append(news_title)

    except Exception as e:
        failed_item.append('기업 관련 뉴스')

    final_return_dict = {}
    if fin_statement is not None and not fin_statement.empty:
        final_return_dict['fin_statement'] = fin_statement
    if fin_report:
        final_return_dict['fin_report'] = fin_report
    if SA_result_corp_rel_news_dict:
        final_return_dict['SA_result_corp_rel_news_dict'] = SA_result_corp_rel_news_dict

    if len(failed_item) > 0:
        print(f"{ticker}에 대해 {failed_item} 항목들의 정보 확인에 실패했습니다.")
        
    return final_return_dict

In [5]:
t1_analyst_common_result = t1_analyst_common(target_year, start_date, end_date)

intl_news 정보 취합 완료!
UK에 대해 Inflation Rate 정보를 찾을 수 없습니다. | [Errno 2] No such file or directory: '/Users/yeonsuk/investment/finTF/pipeline/sub_func/get_info/../../../store_data/raw/FRED/UK/Inflation Rate/2021/2021_Inflation Rate.csv'
--------------------------------------------------
US에 대한 보고서 생성 완료!
--------------------------------------------------
CN에 대한 보고서 생성 완료!
--------------------------------------------------
UK에 대한 보고서 생성 완료!
--------------------------------------------------
JP에 대한 보고서 생성 완료!
--------------------------------------------------
EU에 대한 보고서 생성 완료!


In [6]:
pf_tickers = list(portfolio['Ticker'])
corp_anal_dict = {}
for ticker in pf_tickers:
    t1_analyst_corp_result = t1_analyst_corp(ticker, target_year, start_date, end_date)
    corp_anal_dict[ticker] = t1_analyst_corp_result

재무제표를 불러오는 과정에서 오류가 발생했습니다 | [Errno 2] No such file or directory: '/Users/yeonsuk/investment/finTF/pipeline/sub_func/get_info/../../../store_data/raw/opendart/store_financial_statement/039490/_039490_재무제표 ().csv'
재무제표를 불러오는 과정에서 오류가 발생했습니다 | [Errno 2] No such file or directory: '/Users/yeonsuk/investment/finTF/pipeline/sub_func/get_info/../../../store_data/raw/opendart/store_financial_statement/039490/_039490_재무제표 ().csv'
039490의 이전 년도 재무 데이터를 불러올 수 없습니다.
039490의 fin_statement_info 정보를 확인할 수 없습니다.
재무제표를 불러오는 과정에서 오류가 발생했습니다 | [Errno 2] No such file or directory: '/Users/yeonsuk/investment/finTF/pipeline/sub_func/get_info/../../../store_data/raw/opendart/store_financial_statement/123700/_123700_재무제표 ().csv'
123700에 대해 ['재무제표'] 항목들의 정보 확인에 실패했습니다.
재무제표를 불러오는 과정에서 오류가 발생했습니다 | [Errno 2] No such file or directory: '/Users/yeonsuk/investment/finTF/pipeline/sub_func/get_info/../../../store_data/raw/opendart/store_financial_statement/008260'
재무제표를 불러오는 과정에서 오류가 발생했습니다 | [Errno 2] No such file

In [8]:
system_t1_analyst = """당신은 애널리스트 팀의 리더입니다.
팀원들이 보고서와 함께 주요 정보에 대한 자료를 제공했습니다.
보고받은 자료에 기반해 포트폴리오를 어떻게 조정해야 할지, 오늘의 투자전략에 대한 전반적인 제안서를 작성하세요.
"""

prompt_t1_analyst = f"""애널리스트 보고서: {corp_anal_dict}

현재 포트폴리오: {portfolio}
"""

In [9]:
t1_analyst_response = to_GPT(system_t1_analyst, prompt_t1_analyst)

In [10]:
t1_analyst_response['choices'][0]['message']['content'].split('\n')

['## 투자 전략 제안서',
 '',
 '### 포트폴리오 조정 방향',
 '',
 '현재 포트폴리오 분석 결과, 각 종목의 재무상태와 시장 반응을 종합적으로 고려하여 다음과 같은 조정을 제안합니다.',
 '',
 '### 종목별 평가 및 조정',
 '',
 '1. **008970 (동양철관)**  ',
 '   - **재무 상태**: 손실 지속, 높은 부채비율, 유동부채 급증으로 단기 리스크 증가.',
 '   - **투자 의견**: 보유중인 비중을 줄이거나 매도 고려. 위험 관리 차원에서 최소화 필요.',
 '',
 '2. **039490 (키움증권)**  ',
 '   - **재무 상태**: 영업수익과 순이익 증가, 긍정적인 성장세.',
 '   - **투자 의견**: 매수 추천. 포트폴리오 내 비중 확대 고려.',
 '',
 '3. **021050 (서원)**  ',
 '   - **재무 상태**: 영업이익 증가, 안정적인 자본 구조.',
 '   - **투자 의견**: 유지. 성장 가능성 감안하여 비중 확대 고려.',
 '',
 '4. **058850 (KTcs)**  ',
 '   - **재무 상태**: 영업수익 및 당기순이익 증가, 긍정적인 경영 성과.',
 '   - **투자 의견**: 매수 추천. 비중 확대 가능.',
 '',
 '5. **129260 (인터지스)**  ',
 '   - **재무 상태**: 매출 증가, 당기순이익 흑자 전환.',
 '   - **투자 의견**: 매수 추천. 포트폴리오 내 비중 확대.',
 '',
 '6. **123700 (SJM)**  ',
 '   - **재무 상태**: 자산 및 자본 증가, 영업이익 개선.',
 '   - **투자 의견**: 보유 추천. 안정성 감안하여 유지.',
 '',
 '7. **017040 (광명전기)**  ',
 '   - **재무 상태**: 자본 증가, 안정적인 성장 추세.',
 '   - **투자 의견**: 매수 추천. 비중 확대 고려.',
 '',
 '8. **009180 (한

In [19]:
len(os.listdir('../store_data/raw/market_data/price'))

918

In [20]:
# 10개에 약 60,000토큰
# 918개라면 55,080,000

In [21]:
60000 * 918

55080000